In [94]:
import numpy as np
import pandas as pd
import warnings 
warnings.filterwarnings('ignore')

In [2]:
data = pd.read_excel("completion-detailed-report-11-21-2022-14-34 - Copy.xlsx")

In [3]:
data.head(3)

,User Id,User Status,Module Name,Course Author Id,Course Author Name,Course Type,Primary Tag,Secondary Tag,Started On,Due Date,...,Legal_Entity,Assignment_Reason_Code,Reporting Manager User Id,HR Manager User Id,Module description,Language,Module Description,Module Id,Search Keywords,Course Description
0,99010,Active,Information Security Awareness,AirtelHRLXPadmin,AirtelHRLXP Admin,Classroom,Workshops,-,-,2022-07-29 11:02:43.823,...,-,-,220027,-,Information Security Awareness,English,NaN,4056,NaN,NaN
1,263424,Inactive,Webinar-Demonstrating Resilience as a Soldier,ilearnadmin1,ilearn Admin 1,Self-paced,Internal e-Learning,-,-,2022-04-05 16:21:21.833,...,Bharti Airtel Ltd.,INVOL_NO SHOW,-,220001,Friday Masterclass,English,NaN,7767,NaN,NaN
2,266232,Active,Webinar-Demonstrating Resilience as a Soldier,ilearnadmin1,ilearn Admin 1,Self-paced,Internal e-Learning,-,-,2022-04-05 08:23:56.607,...,Bharti Airtel Ltd.,LEC_ROLCNG,-,206994,Friday Masterclass,English,NaN,7767,NaN,NaN


In [4]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 331558 entries, 0 to 331557
Data columns (total 56 columns):
 #   Column                     Non-Null Count   Dtype         
---  ------                     --------------   -----         
 0   User Id                    331558 non-null  object        
 1   User Status                331558 non-null  object        
 2   Module Name                331558 non-null  object        
 3   Course Author Id           331558 non-null  object        
 4   Course Author Name         331558 non-null  object        
 5   Course Type                331558 non-null  object        
 6   Primary Tag                331558 non-null  object        
 7   Secondary Tag              331558 non-null  object        
 8   Started On                 331558 non-null  object        
 9   Due Date                   331558 non-null  datetime64[ns]
 10  Estimated Duration         331558 non-null  object        
 11  Time Spent                 331558 non-null  object  

In [5]:
data.shape

(331558, 56)

In [6]:
data.describe()

,Completion Percentage,Module Id
count,331558.000000,331558.000000
mean,38.989896,4890.573888
std,47.684037,2044.841744
min,0.000000,103.000000
25%,0.000000,3342.000000
50%,0.000000,5720.000000
75%,100.000000,6079.000000
max,100.000000,8026.000000


In [7]:
data.nunique()

User Id                       23573
User Status                       2
Module Name                    7926
Course Author Id                 17
Course Author Name               11
Course Type                       4
Primary Tag                       7
Secondary Tag                    10
Started On                    96925
Due Date                     140890
Estimated Duration              560
Time Spent                      544
Enrollment Type                   2
Last Accessed On             118130
Course Status                     3
Completion Percentage           292
Enrolled On                  138428
Completed On                  94930
Is Complete                       2
Program Name                     13
Course Name                     281
Assessment Count                 14
Pass Count                       14
Average Score Percentage        456
ROLE_CATEGORY                     5
ORGANIZATION_ID                 852
LOCATION_ID                     394
ROLE                        

In [8]:
#selecting columns which is required for recommendation system

In [9]:
df = data.loc[:,['Course Type','Module Name','Primary Tag','Secondary Tag','Module description']]

In [10]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 331558 entries, 0 to 331557
Data columns (total 5 columns):
 #   Column              Non-Null Count   Dtype 
---  ------              --------------   ----- 
 0   Course Type         331558 non-null  object
 1   Module Name         331558 non-null  object
 2   Primary Tag         331558 non-null  object
 3   Secondary Tag       331558 non-null  object
 4   Module description  303488 non-null  object
dtypes: object(5)
memory usage: 12.6+ MB


In [11]:
df.head(3)

,Course Type,Module Name,Primary Tag,Secondary Tag,Module description
0,Classroom,Information Security Awareness,Workshops,-,Information Security Awareness
1,Self-paced,Webinar-Demonstrating Resilience as a Soldier,Internal e-Learning,-,Friday Masterclass
2,Self-paced,Webinar-Demonstrating Resilience as a Soldier,Internal e-Learning,-,Friday Masterclass


In [12]:
df.isnull().sum()

Course Type               0
Module Name               0
Primary Tag               0
Secondary Tag             0
Module description    28070
dtype: int64

In [13]:
#df['Search Keywords'].fillna('', inplace=True)   

In [14]:
df[['Module description']] = df[['Module description']].fillna('')

In [15]:
#df['Search Keywords'].unique()

In [16]:
df.isnull().sum()

Course Type           0
Module Name           0
Primary Tag           0
Secondary Tag         0
Module description    0
dtype: int64

In [17]:
df.head(3)

,Course Type,Module Name,Primary Tag,Secondary Tag,Module description
0,Classroom,Information Security Awareness,Workshops,-,Information Security Awareness
1,Self-paced,Webinar-Demonstrating Resilience as a Soldier,Internal e-Learning,-,Friday Masterclass
2,Self-paced,Webinar-Demonstrating Resilience as a Soldier,Internal e-Learning,-,Friday Masterclass


In [18]:
duplicatedrows = df[df.duplicated(keep='last')]

In [19]:
df.duplicated(subset=None, keep='first').sum()

323598

In [20]:
df.drop_duplicates(inplace= True)

In [21]:
df.nunique()

Course Type              4
Module Name           7926
Primary Tag              7
Secondary Tag           10
Module description    7533
dtype: int64

In [22]:
#df['Search Keywords']

In [23]:
import nltk
import re
from nltk.corpus import stopwords

In [24]:
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\Srilakshmi\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

# Text data Preprocessing

In [25]:
#combining module_name, module descriptipon search keywords, primary tag and secondary tag create new column new_text

In [26]:
df['new_text'] = df['Module Name'] + ' ' + df['Module description'] + ' ' + ' ' + df['Primary Tag'] + ' ' + df['Secondary Tag']

In [27]:
df.head(3)

,Course Type,Module Name,Primary Tag,Secondary Tag,Module description,new_text
0,Classroom,Information Security Awareness,Workshops,-,Information Security Awareness,Information Security Awareness Information ...
1,Self-paced,Webinar-Demonstrating Resilience as a Soldier,Internal e-Learning,-,Friday Masterclass,Webinar-Demonstrating Resilience as a Soldier ...
10,Self-paced,Work From Home Etiquette,Internal e-Learning,-,Here’s How to Actually Get Stuff Done While Yo...,Work From Home Etiquette Here’s How to Actuall...


In [28]:
df['new_text']

0         Information Security Awareness    Information ...
1         Webinar-Demonstrating Resilience as a Soldier ...
10        Work From Home Etiquette Here’s How to Actuall...
19        aaHRambh TEST HR inductions of all new joiners...
20        SQL Essential Training Learn how to use SQL to...
                                ...                        
330990    Learning Songwriting: Reason Learn how to writ...
331309    Analysing Complexity The first course of the s...
331310    Causal Inference This course offers a rigorous...
331311    Data Management in the Cloud Data Management i...
331351    Python Data Visualization This if the final co...
Name: new_text, Length: 7960, dtype: object

In [29]:
df_new = df.drop(['Primary Tag','Secondary Tag'],axis=1) #after create new column droping the columns

In [30]:
df_new.rename(columns = {'Module Name':'module_name'}, inplace = True)  #rename the variables
df_new.rename(columns = {'Module description':'module_description'}, inplace = True)

In [31]:
df_new

,Course Type,module_name,module_description,new_text
0,Classroom,Information Security Awareness,Information Security Awareness,Information Security Awareness Information ...
1,Self-paced,Webinar-Demonstrating Resilience as a Soldier,Friday Masterclass,Webinar-Demonstrating Resilience as a Soldier ...
10,Self-paced,Work From Home Etiquette,Here’s How to Actually Get Stuff Done While Yo...,Work From Home Etiquette Here’s How to Actuall...
19,Classroom,aaHRambh TEST,HR inductions of all new joiners in HR for all...,aaHRambh TEST HR inductions of all new joiners...
20,LinkedIn Learning,SQL Essential Training,Learn how to use SQL to manage your database. ...,SQL Essential Training Learn how to use SQL to...
...,...,...,...,...
330990,LinkedIn Learning,Learning Songwriting: Reason,"Learn how to write, record, and mix songsâ€”be...",Learning Songwriting: Reason Learn how to writ...
331309,Coursera,Analysing Complexity,The first course of the specialization ANALYZI...,Analysing Complexity The first course of the s...
331310,Coursera,Causal Inference,This course offers a rigorous mathematical sur...,Causal Inference This course offers a rigorous...
331311,Coursera,Data Management in the Cloud,Data Management in the Cloud,Data Management in the Cloud Data Management i...


In [32]:
df_new.shape

(7960, 4)

In [33]:
df_new['new_text'][1] 

'Webinar-Demonstrating Resilience as a Soldier Friday Masterclass  Internal e-Learning -'

In [34]:
df_new['new_text']=df_new['new_text'].apply(str) #convert object to string 

    ***a. Removing non-alpha numerical characters.***
    ***b. Converting all the sentence to lower case***
    ***c. Converting sentence to tokens.***
    ***d. Removing stopwords***
    ***e. Lemmatization of the words***

In [35]:
#NLP tools
import re
import nltk
from nltk.tokenize import word_tokenize
from nltk.stem import WordNetLemmatizer
from gensim.models import Word2Vec
import gensim.downloader as api
from gensim.models import KeyedVectors
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity

In [36]:
en_stopwords =nltk.corpus.stopwords.words('english') 
lemmatizer = WordNetLemmatizer()

In [37]:
#convert new_text column into lower casses
def text_to_lower_case(column,df):
    df[column] = df[column].str.lower()
    return df

In [38]:
df_new = text_to_lower_case('new_text',df_new)

In [39]:
#remove punctuation from new_text column
def remove_punctuation(column,df):
    df[column] = df[column].str.replace('[^\w\s]','')
    return df

In [95]:
df_new = remove_punctuation('new_text',df_new)

In [41]:
#remove numeric from new_text column
def remove_numeric(column,df):
    df[column] = df[column].str.replace('\d+', '')
    return df

In [96]:
df_new = remove_numeric('new_text',df_new)

In [43]:
#REMOVE STOPWORDS
def remove_stopwords(column,df,stopword):
    df[column] = df[column].apply(lambda x: ' '.join(word for word in x.split() if word not in stopword))
    return df

In [44]:
stopword_ls = stopwords.words('english')

In [45]:
df_new = remove_stopwords('new_text',df_new,stopword_ls)

In [46]:
# #word Tokenisation
# def word_tokenize(column,df):
#     df[column] = df[column].apply(lambda x: x.split())
#     return df

In [47]:
# df_new = word_tokenize('new_text',df_new)

In [48]:
df_new.head(3)

,Course Type,module_name,module_description,new_text
0,Classroom,Information Security Awareness,Information Security Awareness,information security awareness information sec...
1,Self-paced,Webinar-Demonstrating Resilience as a Soldier,Friday Masterclass,webinardemonstrating resilience soldier friday...
10,Self-paced,Work From Home Etiquette,Here’s How to Actually Get Stuff Done While Yo...,work home etiquette heres actually get stuff d...


In [49]:
#Defining th preprocessing function tokenization and lemmatization
def process(text):
    tokens = word_tokenize(text)
    clean_list = []
    for token in tokens:
        if token not in en_stopwords:
            clean_list.append(lemmatizer.lemmatize(token))
    return " ".join(clean_list)

In [50]:
df_new.new_text = df_new.new_text.apply(process)

In [51]:
df_new.head(3)

,Course Type,module_name,module_description,new_text
0,Classroom,Information Security Awareness,Information Security Awareness,information security awareness information sec...
1,Self-paced,Webinar-Demonstrating Resilience as a Soldier,Friday Masterclass,webinardemonstrating resilience soldier friday...
10,Self-paced,Work From Home Etiquette,Here’s How to Actually Get Stuff Done While Yo...,work home etiquette here actually get stuff do...


# TF-IDF with Cosine Similarity model

In [52]:
from sklearn.feature_extraction.text import TfidfVectorizer

In [53]:
vectorizer = TfidfVectorizer()

In [54]:
cv = TfidfVectorizer(max_features=5000,stop_words='english') 

In [55]:
tfidfvectors = cv.fit_transform(df_new['new_text']).toarray()

In [56]:
test_matrix = vectorizer.fit_transform(df_new['new_text'])

In [57]:
df_new.head(2)

,Course Type,module_name,module_description,new_text
0,Classroom,Information Security Awareness,Information Security Awareness,information security awareness information sec...
1,Self-paced,Webinar-Demonstrating Resilience as a Soldier,Friday Masterclass,webinardemonstrating resilience soldier friday...


In [58]:
df_new.reset_index(inplace=True)

In [59]:
from sklearn.metrics.pairwise import cosine_similarity

## Row based  - 1

In [60]:
#define a function that will return the first five recommended courses
def Recommendation_Cosine_similarity(matrix, row_num):
    similarity = cosine_similarity(test_matrix)
    similar_courses = list(enumerate(similarity[row_num]))
    sorted_similar_courses = sorted(similar_courses, key=lambda x:x[1], reverse= True)[:6]#the first element is the course being viewed
#This part will return the description of the recommended courses
    i = 0
    for item in sorted_similar_courses:
        description = df_new[df_new.index == item[0]]["module_name"].values[0]
        recommendations = print(f"{i+1}\t{description}\n")
        i = i + 1
    return recommendations

In [75]:
Recommendation_Cosine_similarity(test_matrix, 25)#the first course is the course we are seeking recommedation for

1	Electrical safety

2	Safety Signage and PPE

3	AutoCAD Electrical: Implementing PLCs

4	Electrical Systems: Reading Drawings and Schematics

5	Electrical Systems: Communications and Data

6	Road Safety



## row based - 2

In [102]:
vectorizer = TfidfVectorizer()
features   = vectorizer.fit_transform(df_new['new_text'])

In [103]:
#Taking the cosine similarity of the vectorized text
cosine_sim = cosine_similarity(features,features)

In [104]:
def get_recommended(module_index, cosine_sim):
    module_name = df_new.module_name[module_index]
    indices = pd.Series(df_new.index, index=df_new['module_name']).drop_duplicates()
    # Get index of the df_new
    idx = indices[module_name]
    # Sort the new_text on the similarity scores
    sim_scores = list(enumerate(cosine_sim[idx]))
    sim_scores = sorted(sim_scores, key=lambda x: x[1], reverse=True)
    # Get the scores for 10 most similar module_names
    sim_scores = sim_scores[1:11]
    # Get the module indices
    scores=[i[1] for i in sim_scores]
    news_indices = [i[0] for i in sim_scores]
    # Return the top 10 most similar modules
    dfs=pd.DataFrame({'modulues': df_new['module_name'].iloc[news_indices],
                'Cosine_similarity':  scores})
    print("*" * 10 + " module recommendation " + "*" * 10)
    print("")
    print(f"module name: {module_name}")
    print("")
    print( "*" * 9 + " Recommended modules " + "*" * 9)
    return dfs.iloc[0:,]

In [107]:
df_new.module_name[22]

'Digital Learning : Engineering '

In [108]:
pd.set_option('display.max_colwidth', -1) #sets the maximum width of columns. Cells of this length or longer will be truncated with an ellipsis.

In [110]:
recommended=get_recommended(22, cosine_sim)
recommended

********** module recommendation **********

module name: Digital Learning : Engineering 

********* Recommended modules *********


,modulues,Cosine_similarity
2129,Engineering academy,0.471073
6694,Practical Engineering,0.462796
4171,Tech Soft Skills: Moving from Developer to Engineering Manager,0.399631
3365,Security Awareness: Social Engineering,0.382375
1556,Engineering Manager,0.377343
1298,Software Engineering,0.376355
2633,Applied Machine Learning: Feature Engineering,0.372081
1545,Others - Engineering,0.362904
3991,Cybersecurity Awareness: Social Engineering,0.355318
7015,Engineering Drawings for Manufacturing,0.343792


refferd by kaggle and some websites (analytic vidya,medium,kdnuggts) 

## Based on Module name based

### Approch 1

In [72]:
similarity = cosine_similarity(tfidfvectors)

In [112]:
similarity

array([[1.        , 0.        , 0.        , ..., 0.        , 0.        ,
        0.        ],
       [0.        , 1.        , 0.09837569, ..., 0.        , 0.        ,
        0.        ],
       [0.        , 0.09837569, 1.        , ..., 0.        , 0.        ,
        0.        ],
       ...,
       [0.        , 0.        , 0.        , ..., 1.        , 0.04754967,
        0.05512731],
       [0.        , 0.        , 0.        , ..., 0.04754967, 1.        ,
        0.17853555],
       [0.        , 0.        , 0.        , ..., 0.05512731, 0.17853555,
        1.        ]])

In [92]:
def recommend(module):
    course_index = df_new[df_new['module_name'] == module].index[0]
    distances = similarity[course_index]
    course_list = sorted(list(enumerate(distances)),reverse=True, key=lambda x:x[1])[0:7]
    
    for i in course_list:
        print(df_new.iloc[i[0]].module_name)

In [93]:
recommend('Work From Home Etiquette') 

Work From Home Etiquette
Homes B1 & Below
Building Relationships While Working from Home
B1 & Below |Homes Center
B2 & Above |Homes Center
Program Manager - One Homes
B1 & Below |Homes Circle


### Approch 2

In [83]:
# Get module name/Index
course_indices = pd.Series(df_new.index,index=df_new['module_name']).drop_duplicates()


In [86]:
def recommend_course(title,num_of_rec=10):
    # ID for title
    idx = course_indices[title]
    # Course Indice
    # Search inside cosine_sim_mat
    scores = list(enumerate(similarity[idx]))
    # Scores
    # Sort Scores
    sorted_scores = sorted(scores,key=lambda x:x[1],reverse=True)
    # Recomm
    selected_course_indices = [i[0] for i in sorted_scores[0:]]
    selected_course_scores = [i[1] for i in sorted_scores[0:]]
    result = df_new['module_name'].iloc[selected_course_indices]
    rec_df = pd.DataFrame(result)
    rec_df['similarity_scores'] = selected_course_scores
    return rec_df.head(num_of_rec) 
    

In [115]:
recommend_course('Python Data Visualization',20)

,module_name,similarity_scores
7959,Python Data Visualization,1.000000
5672,Python Data Representations,0.560349
5851,Python Data Analysis,0.540845
926,Python Data Analysis,0.540802
2950,Python Programming Essentials,0.505277
2214,Introduction to Data Science in Python,0.458371
2784,"Python for Data Science Tips, Tricks, & Techniques",0.450642
2216,Python: Data Analysis,0.449418
1217,Python for Data Visualization,0.448987
919,Python for Data Science Essential Training Part 1,0.447598
